## Hypothesis Testing Assignment

In [1]:
# importing necessary packages
import pandas as pd
import numpy as np
import datetime as dt
# package with hypothesis tests
import scipy.stats as st

### Data

You can download the data from [**here**](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing&resourcekey=0-16O9Fc5eaJH99-M7AHqHOg). The data contains results of all NBA games from seasons 2013/2014 to 2015/2016.

In [2]:
data = pd.read_csv("data/nba_games_2013_2015.csv", delimiter=";")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7380 entries, 0 to 7379
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          7380 non-null   int64  
 1   TEAM_ID            7380 non-null   int64  
 2   TEAM_ABBREVIATION  7380 non-null   object 
 3   TEAM_NAME          7380 non-null   object 
 4   GAME_ID            7380 non-null   int64  
 5   GAME_DATE          7380 non-null   object 
 6   MATCHUP            7380 non-null   object 
 7   WL                 7380 non-null   object 
 8   MIN                7380 non-null   int64  
 9   PTS                7380 non-null   int64  
 10  FGM                7380 non-null   int64  
 11  FGA                7380 non-null   int64  
 12  FG_PCT             7380 non-null   float64
 13  FG3M               7380 non-null   int64  
 14  FG3A               7380 non-null   int64  
 15  FG3_PCT            7380 non-null   float64
 16  FTM                7380 

In [3]:
data["SEASON_ID"].unique()

array([22015, 22014, 22013])

In [4]:
data["GAME_DATE"].unique()

array(['2016-04-13', '2016-04-12', '2016-04-11', '2016-04-10',
       '2016-04-09', '2016-04-08', '2016-04-07', '2016-04-06',
       '2016-04-05', '2016-04-03', '2016-04-02', '2016-04-01',
       '2016-03-31', '2016-03-30', '2016-03-29', '2016-03-28',
       '2016-03-27', '2016-03-26', '2016-03-25', '2016-03-24',
       '2016-03-23', '2016-03-22', '2016-03-21', '2016-03-20',
       '2016-03-19', '2016-03-18', '2016-03-17', '2016-03-16',
       '2016-03-15', '2016-03-14', '2016-03-13', '2016-03-12',
       '2016-03-11', '2016-03-10', '2016-03-09', '2016-03-08',
       '2016-03-07', '2016-03-06', '2016-03-05', '2016-03-04',
       '2016-03-03', '2016-03-02', '2016-03-01', '2016-02-29',
       '2016-02-28', '2016-02-27', '2016-02-26', '2016-02-25',
       '2016-02-24', '2016-02-23', '2016-02-22', '2016-02-21',
       '2016-02-20', '2016-02-19', '2016-02-18', '2016-02-11',
       '2016-02-10', '2016-02-09', '2016-02-08', '2016-02-07',
       '2016-02-06', '2016-02-05', '2016-02-04', '2016-

--------------
### Task 1
Split the data into **3** separate dataframes for each NBA season!

In [5]:
data_2013 = data[data["SEASON_ID"] == 22013]
data_2014 = data[data["SEASON_ID"] == 22014]
data_2015 = data[data["SEASON_ID"] == 22015]

In [6]:
print(f"There are {len(data_2013)} games recorded in the 2013/2014 season.")
print(f"There are {len(data_2014)} games recorded in the 2014/2015 season.")
print(f"There are {len(data_2015)} games recorded in the 2015/2016 season.")

There are 2460 games recorded in the 2013/2014 season.
There are 2460 games recorded in the 2014/2015 season.
There are 2460 games recorded in the 2015/2016 season.


---------------
### Task 2
Test the hypothesis that the offensive productions stats of the Cleveland Cavaliers and Golden State Warriors (the teams that met in the finals that year) were from the same distribution in the 2015/2016 season.

Offensive production refers to two variables: **PTS (Points)** and **FG_PCT (Field Goal Percentage)**. We will need to do two separate hypothesis tests, one for each variable.

#### Team Data

In [7]:
data_2015["TEAM_ID"].unique()

array([1610612750, 1610612749, 1610612738, 1610612747, 1610612739,
       1610612758, 1610612746, 1610612764, 1610612737, 1610612761,
       1610612740, 1610612753, 1610612762, 1610612744, 1610612741,
       1610612748, 1610612763, 1610612759, 1610612766, 1610612751,
       1610612742, 1610612745, 1610612755, 1610612757, 1610612756,
       1610612743, 1610612754, 1610612765, 1610612752, 1610612760])

In [8]:
data_2015["TEAM_NAME"].unique()

array(['Minnesota Timberwolves', 'Milwaukee Bucks', 'Boston Celtics',
       'Los Angeles Lakers', 'Cleveland Cavaliers', 'Sacramento Kings',
       'LA Clippers', 'Washington Wizards', 'Atlanta Hawks',
       'Toronto Raptors', 'New Orleans Pelicans', 'Orlando Magic',
       'Utah Jazz', 'Golden State Warriors', 'Chicago Bulls',
       'Miami Heat', 'Memphis Grizzlies', 'San Antonio Spurs',
       'Charlotte Hornets', 'Brooklyn Nets', 'Dallas Mavericks',
       'Houston Rockets', 'Philadelphia 76ers', 'Portland Trail Blazers',
       'Phoenix Suns', 'Denver Nuggets', 'Indiana Pacers',
       'Detroit Pistons', 'New York Knicks', 'Oklahoma City Thunder'],
      dtype=object)

In [9]:
data_2015["TEAM_ABBREVIATION"].unique()

array(['MIN', 'MIL', 'BOS', 'LAL', 'CLE', 'SAC', 'LAC', 'WAS', 'ATL',
       'TOR', 'NOP', 'ORL', 'UTA', 'GSW', 'CHI', 'MIA', 'MEM', 'SAS',
       'CHA', 'BKN', 'DAL', 'HOU', 'PHI', 'POR', 'PHX', 'DEN', 'IND',
       'DET', 'NYK', 'OKC'], dtype=object)

##### Isolate Cleveland and Warriors Stats

In [10]:
data_2015_cavs = data_2015[data_2015["TEAM_ABBREVIATION"] == "CLE"]
data_2015_wars = data_2015[data_2015["TEAM_ABBREVIATION"] == "GSW"]

In [11]:
len(data[data["TEAM_ABBREVIATION"] == "CLE"])

246

#### Hypothesis Testing for `Points` (PTS)
If the Cavs and the Warriors offensive production stats are from the same distribution in the 2015/2016 seasons, then the distribution of points should be similar.

To do this, we will compare the two distributions against each other using the t-test, assuming that $$H_0: \mu_{cavs} = \mu_{wars}$$

We will using a pair-wise $t-test$, which is calculated as follows: $$t = \frac{\bar(x_1) - \bar(x_2)}{\sigma{\sqrt{\frac{1}{n_1} + \frac{1}{n_2}}}}$$

A large $t$-value will reveal that the difference between groups is larger than the pooled standard error, indicating a more significant difference between the groups. But first, is it safe to assume that the standard deviations between the two teams are equal?

##### Levene's Test for Equality of Variances, $H0: s_1 = s_2$
For this test, we will set the confidence level at $\alpha =$ 0.05

In [12]:
ttest, pval_pts = st.levene(data_2015_cavs["PTS"], data_2015_wars["PTS"])
print(f"The resulting p-value for Levene's test is {pval_pts:.2f} which is greater than the specified alpha value, 0.05.")
print("This suggests that the difference between the standard deviations of the two teams' PTS is insignificant.")

The resulting p-value for Levene's test is 0.89 which is greater than the specified alpha value, 0.05.
This suggests that the difference between the standard deviations of the two teams' PTS is insignificant.


##### Finding $\bar{x_1}$ and $\bar{x_2}$

In [13]:
# for the Cleveland Cavaliers, x1
cavs_2015_pts_mean = st.tmean(data_2015_cavs["PTS"])
# for the Golden State Warriors, x2
wars_2015_pts_mean = st.tmean(data_2015_wars["PTS"])

##### Finding $s_1$ and $s_2$ to determine $\sigma$ (*pooled standard variance*)

In [14]:
# for the Cleveland Cavaliers
cavs_2015_pts_std = st.tstd(data_2015_cavs["PTS"])
# for the Golden State Warriors
wars_2015_pts_std = st.tstd(data_2015_wars["PTS"])
# determine pooled standard variance
pts_sigma = st.tmean([cavs_2015_pts_std, wars_2015_pts_std])
pts_sigma

11.315425853726083

##### Finding $n_1$ and $n_2$

In [15]:
# for the Cleveland Cavaliers
cavs_2015_n = len(data_2015_cavs)
# for the Golden State Warriors
wars_2015_n = len(data_2015_wars)

##### Finding t-test values

In [16]:
tstat_from_calc = (cavs_2015_pts_mean - wars_2015_pts_mean) / (pts_sigma * np.sqrt(1/cavs_2015_n + 1/wars_2015_n))
tstat_from_calc

-5.976199196736252

##### Using *scipy* ttest

In [17]:
tstat_from_st, pvalue = st.ttest_ind(data_2015_cavs["PTS"], data_2015_wars["PTS"])

In [18]:
print(f"The t-statistic from the calculations: {tstat_from_calc:.2f}")
print(f"The t-statistic from the scipy module: {tstat_from_st:.2f}")

The t-statistic from the calculations: -5.98
The t-statistic from the scipy module: -5.97


In [19]:
print(f"The p-value rejects the null hypothesis that the points scored by the Cavs and the Warriors in the 2015/2016 season are from the same distribution.")

The p-value rejects the null hypothesis that the points scored by the Cavs and the Warriors in the 2015/2016 season are from the same distribution.


#### Hypothesis Testing for `Field Goal Percentage` (FG_PCT)

##### Levene's Test for Equality of Variances

In [20]:
_, pval_fgpct = st.levene(data_2015_cavs["FG_PCT"], data_2015_wars["FG_PCT"])
pval_fgpct
print(f"The resulting p-value for Levene's test is {pval_fgpct:.2f} which is greater than the specified alpha value, 0.05.")
print("This suggests that the difference between the standard deviations of the two teams' field goal percentages is insignificant.")

The resulting p-value for Levene's test is 0.42 which is greater than the specified alpha value, 0.05.
This suggests that the difference between the standard deviations of the two teams' field goal percentages is insignificant.


##### Finding t-test values

In [21]:
_, pvalue = st.ttest_ind(data_2015_cavs["FG_PCT"], data_2015_wars["FG_PCT"])
print(pvalue)

0.00206097581047554


---
### Task 3
Test the hypothesis that the number of points (PTS) scored by Cleveland Cavaliers changed significantly after the head coach changed in the 2015/2016 season.

- **Coach Blatt was fired on 24th of Jan, 2016**. 

Use the data from seasons 2014/2015 and 2015/2016 - those are years when Cleveland was coached by Blatt. 

**We have two possible solutions to try here:**
- Take the same amount of games from before and after and try t-test.
- Take all the games from before and after and look for the right test to compare two samples with different sizes. (You will need to go through the scipy documentation or google to figure out what kind of test is required.)

In [22]:
print(len(data_2014) + len(data_2015))
data_2seasons = pd.concat([data_2014, data_2015])

4920


#### Convert Dates to `datetime` Objects

In [23]:
data_2seasons["GAME_DATE"] = data_2seasons["GAME_DATE"].apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%d").date())

In [24]:
print(f"The NBA hosted a total of {len(data_2seasons)} games in the 2014/2015 and 2015/2016 seasons.")

The NBA hosted a total of 4920 games in the 2014/2015 and 2015/2016 seasons.


#### Isolate `Cleveland`

In [25]:
data_cle = data_2seasons[data_2seasons["TEAM_ABBREVIATION"] == "CLE"]

In [26]:
print(f"The Cleveland Cavaliers played a total of {len(data_cle)} games in the 2014/2015 and 2015/2016 seasons.")

The Cleveland Cavaliers played a total of 164 games in the 2014/2015 and 2015/2016 seasons.


#### Use `GAME_DATE` to Divide Games Before and After Blatt

In [27]:
data_cle_after_blatt = data_cle[data_cle["GAME_DATE"] > dt.date(2016, 1, 24)]
data_cle_before_blatt = data_cle[data_cle["GAME_DATE"] <= dt.date(2016, 1, 24)]

In [28]:
data_cle_before_blatt.sort_values(by="GAME_DATE").head(10)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
4881,22014,1610612739,CLE,Cleveland Cavaliers,21400018,2014-10-30,CLE vs. NYK,L,240,90,...,0.850,11,22,33,22,5,2,19,15,-5.0
4877,22014,1610612739,CLE,Cleveland Cavaliers,21400022,2014-10-31,CLE @ CHI,W,265,114,...,0.879,20,32,52,18,8,4,9,18,6.0
4815,22014,1610612739,CLE,Cleveland Cavaliers,21400055,2014-11-04,CLE @ POR,L,241,82,...,0.786,13,28,41,18,5,3,14,19,-19.0
4792,22014,1610612739,CLE,Cleveland Cavaliers,21400066,2014-11-05,CLE @ UTA,L,241,100,...,0.850,7,23,30,6,8,4,12,22,-2.0
4779,22014,1610612739,CLE,Cleveland Cavaliers,21400081,2014-11-07,CLE @ DEN,W,240,110,...,0.727,6,36,42,25,5,3,8,14,9.0
4725,22014,1610612739,CLE,Cleveland Cavaliers,21400098,2014-11-10,CLE vs. NOP,W,241,118,...,0.833,12,33,45,26,3,5,9,13,7.0
4659,22014,1610612739,CLE,Cleveland Cavaliers,21400125,2014-11-14,CLE @ BOS,W,240,122,...,0.757,8,32,40,22,5,5,11,21,1.0
4657,22014,1610612739,CLE,Cleveland Cavaliers,21400133,2014-11-15,CLE vs. ATL,W,239,127,...,0.769,12,35,47,39,12,2,13,14,33.0
4614,22014,1610612739,CLE,Cleveland Cavaliers,21400147,2014-11-17,CLE vs. DEN,L,239,97,...,0.826,9,33,42,18,7,8,15,15,-9.0
4600,22014,1610612739,CLE,Cleveland Cavaliers,21400159,2014-11-19,CLE vs. SAS,L,239,90,...,0.692,10,28,38,23,7,5,16,21,-2.0


In [29]:
data_cle_after_blatt.sort_values(by="GAME_DATE").head(10)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1129,22015,1610612739,CLE,Cleveland Cavaliers,21500667,2016-01-25,CLE vs. MIN,W,239,114,...,0.650,12,31,43,30,7,3,15,18,7.0
1083,22015,1610612739,CLE,Cleveland Cavaliers,21500684,2016-01-27,CLE vs. PHX,W,239,115,...,0.875,10,27,37,34,12,3,13,17,22.0
1055,22015,1610612739,CLE,Cleveland Cavaliers,21500699,2016-01-29,CLE @ DET,W,240,114,...,0.931,10,37,47,20,4,1,7,25,8.0
1038,22015,1610612739,CLE,Cleveland Cavaliers,21500713,2016-01-30,CLE vs. SAS,W,240,117,...,0.857,10,33,43,24,4,4,9,21,14.0
1017,22015,1610612739,CLE,Cleveland Cavaliers,21500721,2016-02-01,CLE @ IND,W,265,111,...,0.696,10,35,45,22,8,3,16,19,5.0
983,22015,1610612739,CLE,Cleveland Cavaliers,21500735,2016-02-03,CLE @ CHA,L,239,97,...,0.786,3,25,28,16,9,5,7,19,-9.0
958,22015,1610612739,CLE,Cleveland Cavaliers,21500755,2016-02-05,CLE vs. BOS,L,240,103,...,0.600,15,32,47,21,8,6,14,25,-1.0
941,22015,1610612739,CLE,Cleveland Cavaliers,21500763,2016-02-06,CLE vs. NOP,W,239,99,...,1.000,10,39,49,21,6,6,12,17,15.0
905,22015,1610612739,CLE,Cleveland Cavaliers,21500775,2016-02-08,CLE vs. SAC,W,241,120,...,0.737,12,41,53,27,7,5,14,25,20.0
875,22015,1610612739,CLE,Cleveland Cavaliers,21500795,2016-02-10,CLE vs. LAL,W,241,120,...,0.611,13,33,46,26,6,1,6,20,9.0


#### Levene's Test for Equality of Variances

In [30]:
_, pval_blatt = st.levene(data_cle_before_blatt["PTS"], data_cle_after_blatt["PTS"])
print(f"Given a significance level of 0.05, the p-value of {pval_blatt:.2f} rejects the hypothesis that the points scored before and after Blatt do not vary significantly.")

Given a significance level of 0.05, the p-value of 0.02 rejects the hypothesis that the points scored before and after Blatt do not vary significantly.


#### Find `t-statistic` and `p-value`
Given that the standard deviation of each sample set cannot be pooled, unequal variance must be specified in the `ttest_ind` to perform a Welch's t-test.

In [31]:
_, pval_blatt = st.ttest_ind(data_cle_before_blatt["PTS"], data_cle_after_blatt["PTS"], equal_var=False)
print(f"Given a significance level of 0.05, the p-value of {pval_blatt:.2f} resulting from the t-test comparing the PTS (points) data for the Cleveland Cavaliers before and after Blatt rejects the hypothesis that the two distributions are similarly distributed.")

Given a significance level of 0.05, the p-value of 0.00 resulting from the t-test comparing the PTS (points) data for the Cleveland Cavaliers before and after Blatt rejects the hypothesis that the two distributions are similarly distributed.


----------------


### Task 4
Download [**the similar dataset**](https://drive.google.com/file/d/0Bz9_0VdXvv9bRHhuRTI1aXBQcTA/view?usp=sharing&resourcekey=0-jGIaWyk0bXyECNSzB3x38w) with scores from playoff games in 2016.

In [32]:
data_playoffs = pd.read_csv("data/nba_playoff_games_2016.csv", delimiter=";")
data_playoffs

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42015,1610612739,CLE,Cleveland Cavaliers,41500407,2016-06-19,CLE @ GSW,W,241,93,...,0.840,9,39,48,17,7,6,11,15,4.0
1,42015,1610612744,GSW,Golden State Warriors,41500407,2016-06-19,GSW vs. CLE,L,239,89,...,0.769,7,32,39,22,7,5,10,23,-4.0
2,42015,1610612744,GSW,Golden State Warriors,41500406,2016-06-16,GSW @ CLE,L,238,101,...,0.690,9,26,35,19,5,3,14,25,-14.0
3,42015,1610612739,CLE,Cleveland Cavaliers,41500406,2016-06-16,CLE vs. GSW,W,240,115,...,0.781,8,37,45,24,12,7,10,25,14.0
4,42015,1610612739,CLE,Cleveland Cavaliers,41500405,2016-06-13,CLE @ GSW,W,241,112,...,0.609,8,33,41,15,11,9,16,22,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,42015,1610612760,OKC,Oklahoma City Thunder,41500161,2016-04-16,OKC vs. DAL,W,243,108,...,0.857,14,42,56,23,7,6,18,25,38.0
168,42015,1610612737,ATL,Atlanta Hawks,41500131,2016-04-16,ATL vs. BOS,W,238,102,...,0.771,13,40,53,23,4,9,11,20,1.0
169,42015,1610612745,HOU,Houston Rockets,41500141,2016-04-16,HOU @ GSW,L,240,78,...,0.500,13,38,51,16,10,6,24,28,-26.0
170,42015,1610612742,DAL,Dallas Mavericks,41500161,2016-04-16,DAL @ OKC,L,238,70,...,0.615,9,24,33,16,5,0,12,25,-38.0


------------
### Task 5
Test the hypothesis that **number of blocks (BLK)** are from the same distribution in both the NBA playoffs and in the NBA regular season for 2015/2016 seaon for the **Toronto Raptors**. 

- We will be working with two samples with different sizes again.

#### Toronto Raptors Data

In [33]:
data_raps_2015 = data_2015[data_2015["TEAM_ABBREVIATION"] == "TOR"]
data_raps_playoffs = data_playoffs[data_playoffs["TEAM_ABBREVIATION"] == "TOR"]

In [34]:
raps_blk_2015 = data_raps_2015["BLK"]
raps_blk_playoffs = data_raps_playoffs["BLK"]

#### Levene's Test for Equality of Variances

In [35]:
stat_blk, pval_blk = st.levene(raps_blk_2015, raps_blk_playoffs)
print(f"Given a confidence level of 0.05, the p-value of {pval_blk:.2f} calculated using Levene's test for equality suggests that the null hypothesis can be assumed to be correct. The variances between the Raptors' regular and off-season block stats can be equivalent.")

Given a confidence level of 0.05, the p-value of 0.09 calculated using Levene's test for equality suggests that the null hypothesis can be assumed to be correct. The variances between the Raptors' regular and off-season block stats can be equivalent.


#### `t-test`

In [36]:
# the default value for `equal_var' is True so it is not specified here`
ttest_stat_blk, ttest_pval_blk = st.ttest_ind(raps_blk_2015, raps_blk_playoffs)
print(f"Given a confidence level of 0.05, the p-value of {ttest_pval_blk:.2f} rejects the null hypothesis that the block stats for the Raptors are similar between the regular and off-season in 2015. They can be considered ")

Given a confidence level of 0.05, the p-value of 0.01 rejects the null hypothesis that the block stats for the Raptors are similar between the regular and off-season in 2015. They can be considered 


In [37]:
ttest1s_stat_blk, ttest1s_pval_blk = st.ttest_ind(raps_blk_2015, raps_blk_playoffs, alternative="greater")
print(f"Furthermore, a p-value of {ttest1s_pval_blk:.2f} suggests the rejection of the null hypothesis, which implies that the one-sided alternative is that the Raptors' mean block stat for the regular season is greater than their off-season.")


Furthermore, a p-value of 0.00 suggests the rejection of the null hypothesis, which implies that the one-sided alternative is that the Raptors' mean block stat for the regular season is greater than their off-season.



-----------------
### Task 6
Test the hypothesis that the number of points (PTS) scored by Cleveland Cavaliers is equally distributed for all 3 seasons. 

- In this case, we need a hypothesis test that compares more than 2 distributions at the same. (You will need to go through the scipy documentation or google to figure out what kind of test is required.)

In [39]:
# isolate Cleveland Cavaliers for all seasons
data_cavs_2013 = data_2013[data_2013["TEAM_ABBREVIATION"] == "CLE"]
data_cavs_2014 = data_2014[data_2014["TEAM_ABBREVIATION"] == "CLE"]
data_cavs_2015 = data_2015[data_2015["TEAM_ABBREVIATION"] == "CLE"]

#### One-Way ANOVA Testing

In [42]:
# one-way ANOVA testing
one_way_stat, one_way_pval = st.f_oneway(data_cavs_2013["PTS"], data_cavs_2014["PTS"], data_cavs_2015["PTS"])
one_way_pval

In [46]:
print(f"The one-way ANOVA test resulted in a p-value of {one_way_pval:.2f}, which is less than the confidence level of 0.05.")
print(f"Thus, the null hypothesis that the Cavs points stats between the 3 seasons do not follow the same distribution.")

The one-way ANOVA test resulted in a p-value of 0.00, which is less than the confidence level of 0.05.
Thus, the null hypothesis that the Cavs points stats between the 3 seasons do not follow the same distribution.


#### Follow Up
**Between which seasons can we see the significant difference?**

+ Unfortunatelly, this is not the output of an ANOVA test and further tests are needed to be run.
+ Note: Lebron James came back to the Cleveland Caveliers prior to season 2014/2015. We can use this fact to interpret our results correctly.

#### Levene's Test for Equality of Variances

In [51]:
# to solve this problem, we need to run hypothesis testing between each consecutive season
# but first, we need to determine that the standard deviations are similar enough to assume conformity with the ttest
_, cavs_2013_2014_pval = st.levene(data_cavs_2013["PTS"], data_cavs_2014["PTS"])
_, cavs_2014_2015_pval = st.levene(data_cavs_2014["PTS"], data_cavs_2015["PTS"])

print(f"The p-values comparing the standard deviations between the 2013/2014 and 2014/2015 seasons are:\n 2013/2014: {cavs_2013_2014_pval:.2f}\n 2014/2015: {cavs_2014_2015_pval:.2f}")
print(f"Given a significance level of 0.05, both comparisons meet the requirement of the null hypothesis and can be assumed to be similar.")

The p-values comparing the standard deviations between the 2013/2014 and 2014/2015 seasons are:
 2013/2014: 0.74
 2014/2015: 0.23
Given a significance level of 0.05, both comparisons meet the requirement of the null hypothesis and can be assumed to be similar.


#### two-sided `t-test`

In [55]:
_, pval_2013_2014 = st.ttest_ind(data_cavs_2013["PTS"], data_cavs_2014["PTS"])
_, pval_2014_2015 = st.ttest_ind(data_cavs_2014["PTS"], data_cavs_2015["PTS"])

In [58]:
print(f"With a p-value of {pval_2013_2014:.2f}, the 2013/2014 season points stats are different enough to be considered significantly so.")

With a p-value of 0.01, the 2013/2014 season points stats are different enough to be considered significantly so.


In [59]:
print(f"With a p-value of {pval_2014_2015:.2f}, the 2014/2015 season points stats cannot reject the null hypothesis.")

With a p-value of 0.52, the 2014/2015 season points stats cannot reject the null hypothesis.
